# Úloha č. 2: Test hypotézy

Autor: *Natália Holková*

Login: *xholko02*



## Zadanie

Overte nasledujúcu hypotézu s 95% istotou:

*Pokiaľ bol vinník nehody pod silným vplyvom alkoholu, došlo častejšie k ťažkým zdravotným následkom.*

Využite **$\chi^2$** test.

In [ ]:
###
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats import chi2

## Načítanie a vyčistenie dát

In [ ]:
# Načítanie dát
df = pd.read_pickle("accidents.pkl.gz")

### Štruktúra stĺpcov *p11*, *p13a* a *p13b*

#### *p11*
Predstavuje prítomnosť alkoholu u vinníka nehody. Povolené sú celé čísla z intervalu **<0,9>**.

| Hodnota | Alkohol |
|---|---|
| 0 | nezisťované |
| 1 | áno, < 0.24 ‰ |
| 2 | nie |
| 3 | áno, >= 0.24 & < 0.5 ‰ |
| 4 | pod vplyvom drog |
| 5 | pod vplyvom alkoholu a drog |
| 6 | áno, >= 0.5 & < 0.8 ‰ |
| 7 | áno, >= 0.8 & < 1.0 ‰ |
| 8 | áno, >= 1.0 & < 1.5 ‰ |
| 9 | áno, >= 1.5 ‰ |

Za silný vplyv alkoholu budeme považovať iba stavy, keď ***p11* >= 7 ** a vylúčime nehody pod vplyvom drog (***p11* = 4** a ***p11* = 5**).


#### *p13a*
Predstavuje počet osôb, ktoré boli usmrtené pri nehode.
Hodnota musí byť celé číslo **>= 0**.

#### *p13b*
Predstavuje počet osôb, ktoré boli ťažko zranené pri nehode.
Hodnota musí byť celé číslo **>= 0**.

Z celého datasetu vylúčime nehody pod vplyvom drog. 

In [ ]:
df = df.query('p11 >= 0 & p11 <= 9 & p11 != 4 & p11 != 5')

## $\chi^2$ test

Urobíme binarizáciu premenných.

Vytvoríme nový DataFrame s informáciou či bol vinník pod silným vplyvom alkoholu (*p11 >= 7*) a ku koľkým ťažkým zdravotným následkom došlo (*p13a + p13b*).

In [ ]:
# binarizácia
df_bis = pd.DataFrame({
          'strong_alcohol_influence': (df['p11'] >= 7).astype(int),
          'total_serious_injuries': df['p13a'] + df['p13b']
         }).dropna()  # vyradiť prázdne

Vytvoríme kontingenčnú tabulku.

In [ ]:
# kontingenčná tabulka
ct = pd.crosstab(df_bis['strong_alcohol_influence'], df_bis['total_serious_injuries'])
ct ###

Spočítame chí-kvadrát štatistiku a p-hodnotu pre test nezávislosti v kontingenčnej tabulke. Táto funkcia vráti viac hodnot, nás zaujímá druhá hodnota (p-hodnota).

In [ ]:
stat, p, dof, expected = chi2_contingency(ct)
print(f'p = {p}') # p-hodnota

p-hodnota je pravdepodobnosť získania výsledkov minimálne tak extrémnych, ako sú pozorované výsledky testu štatistických hypotéz, za predpokladu, že nulová hypotéza je správna.

p-hodnota je miera pravdepodobnosti, že by sa pozorovaný rozdiel mohol vyskytnúť iba náhodou. Čím je nižšia p-hodnota, tý je väčšia štatistická významnosť pozorovaného rozdielu.


In [ ]:
# interpretovať p-hodnotu
prob = 0.95 # 95% istota
alpha = 1.0 - prob
if p <= alpha:
	print('Zamietnuť nulovú hypotézu')
else:
	print('Nepodarilo sa zamietnuť nulovú hypotézu')

## Záver

Na základe výsledkov sme vyvrátili sme nulovú hypotézu.